In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [2]:
channels = ['AF3', 'AF4', 'F3', 'F4', 'F7', 'F8', 'FC5', 'FC6', 'O1', 'O2', 'P7', 'P8', 'T7', 'T8']

In [3]:
epochs_folder = 'gameemo_database_epochs'
epochs_psd_folder = 'gameemo_database_epochs_psd'
labels_path = 'GameLabels\GAMEEMO_SCORES.xlsx'
labels_sheet = 'All'
labels_file = pd.read_excel(labels_path, labels_sheet)
labels_file

,Subject,Boring,Horrible,Calm,Funny
0,1,8,1,3,2
1,1,2,1,8,8
2,1,2,8,1,1
3,1,1,1,3,9
4,2,8,1,4,2
...,...,...,...,...,...
107,27,1,1,7,5
108,28,7,1,6,1
109,28,1,1,8,8
110,28,1,7,6,1


In [4]:
labels_array = np.array(labels_file[['Boring', 'Horrible', 'Calm', 'Funny']])
labels_array

array([[ 8,  1,  3,  2],
       [ 2,  1,  8,  8],
       [ 2,  8,  1,  1],
       [ 1,  1,  3,  9],
       [ 8,  1,  4,  2],
       [ 2,  1,  7,  6],
       [ 2,  8,  3,  2],
       [ 2,  2,  7,  9],
       [ 7,  2,  6,  1],
       [ 1,  1,  8,  8],
       [ 1, 10,  1,  1],
       [ 1,  1,  3,  9],
       [ 8,  1,  3,  1],
       [ 3,  1,  9,  6],
       [ 1,  9,  2,  1],
       [ 1,  1,  3,  9],
       [ 7,  1,  2,  1],
       [ 2,  1,  5,  7],
       [ 1,  8,  1,  2],
       [ 2,  1,  3,  8],
       [ 7,  1,  3,  1],
       [ 1,  1,  7,  3],
       [ 1,  8,  1,  1],
       [ 1,  1,  8,  8],
       [ 8,  1,  3,  3],
       [ 3,  1,  8,  6],
       [ 1, 10,  1,  1],
       [ 1,  1,  1, 10],
       [10,  1,  1,  1],
       [ 3,  1,  8,  8],
       [ 1,  5,  7,  1],
       [ 1,  1,  6,  6],
       [10,  1,  4,  4],
       [ 1,  1,  8,  6],
       [ 1,  9,  2,  3],
       [ 1,  1,  8,  8],
       [ 8,  1,  5,  4],
       [ 1,  1,  7,  2],
       [ 1,  8,  2,  1],
       [ 1,  1,  5, 10],


In [5]:
epochs_data = []
labels_data = []

k=0

for subject in tqdm(os.listdir(epochs_folder)):
    for game in os.listdir(os.path.join(epochs_folder, subject)):
        k+=1
        for epoch in os.listdir(os.path.join(epochs_folder, subject, game)):
            read_epoch = pd.read_csv(os.path.join(epochs_folder, subject, game, epoch))
            read_epoch = read_epoch[channels].T
            epochs_data.append(np.array(read_epoch))
            labels_data.append(labels_array[k-1])



100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [02:09<00:00,  4.62s/it]


In [31]:
epochs_data = []
labels_data = []

k=0

for subject in tqdm(os.listdir(epochs_psd_folder)):
    for game in os.listdir(os.path.join(epochs_psd_folder, subject)):
        k+=1
        for epoch in os.listdir(os.path.join(epochs_psd_folder, subject, game)):
            read_epoch = pd.read_csv(os.path.join(epochs_psd_folder, subject, game, epoch))
            read_epoch = read_epoch[channels].T
            epochs_data.append(np.array(read_epoch))
            labels_data.append(labels_array[k-1])



100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [01:19<00:00,  2.83s/it]


In [6]:
print((epochs_data[0].shape), (labels_data[0].shape))

(14, 641) (4,)


In [7]:
main_data = np.array(epochs_data)
main_labels = np.array(labels_data)
print(main_data.shape, main_labels.shape)

(6608, 14, 641) (6608, 4)


In [8]:
from sklearn.model_selection import train_test_split

# First, split into training+validation and testing sets
X_train_val, X_test, y_train_val, y_test = train_test_split(main_data, main_labels, test_size=0.2, random_state=42)

# Then, split the training+validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Print the shapes of the resulting datasets
print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)
print("Testing set shape:", X_test.shape, y_test.shape)


Training set shape: (3964, 14, 641) (3964, 4)
Validation set shape: (1322, 14, 641) (1322, 4)
Testing set shape: (1322, 14, 641) (1322, 4)


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

In [10]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [11]:
def EEGNet(nb_classes = 4, Chans = 14, Samples = 641, 
             dropoutRate = 0.5, kernLength = 64, F1 = 8, 
             D = 4, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    # input1   = Input(shape = (Chans, Samples, 1))
    # block1       = Conv2D(F1, (1, kernLength), padding = 'same',
    #                                input_shape = (Chans, Samples, 1),
    #                                use_bias = False)(input1)
    # block1       = BatchNormalization()(block1)
    # block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
    #                                depth_multiplier = D,
    #                                depthwise_constraint = max_norm(1.))(block1)
    # block1       = BatchNormalization()(block1)
    # block1       = Activation('elu')(block1)
    # block1       = AveragePooling2D((1, 4))(block1)
    # block1       = dropoutType(dropoutRate)(block1)
    
    # block2       = SeparableConv2D(F2, (1, 16),
    #                                use_bias = False, padding = 'same')(block1)
    # block2       = BatchNormalization()(block2)
    # block2       = Activation('elu')(block2)
    # block2       = AveragePooling2D((1, 8))(block2)
    # block2       = dropoutType(dropoutRate)(block2)
        
    # flatten      = Flatten(name = 'flatten')(block2)
    
    # dense        = Dense(nb_classes, name = 'dense', 
    #                      kernel_constraint = max_norm(norm_rate))(flatten)
    # # softmax      = Activation('softmax', name = 'softmax')(dense)
    # output      = Activation('linear', name = 'linear')(dense)
    # return Model(inputs=input1, outputs=output)

    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(25, (1, 5), 
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    # block1       = Conv2D(25, (Chans, 1),
    #                              kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block1)
    block1       = Activation('elu')(block1)
    block1       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block1)
    block1       = Dropout(dropoutRate)(block1)
  
    block2       = Conv2D(50, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block2       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block2)
    block2       = Activation('elu')(block2)
    block2       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block2)
    block2       = Dropout(dropoutRate)(block2)
    
    block3       = Conv2D(100, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block2)
    block3       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block3)
    block3       = Activation('elu')(block3)
    block3       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block3)
    block3       = Dropout(dropoutRate)(block3)
    
    block4       = Conv2D(200, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block3)
    block4       = Conv2D(25, (Chans, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block4)
    block4       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block4)
    block4       = Activation('elu')(block4)
    block4       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block4)
    block4       = Dropout(dropoutRate)(block4)
    
    flatten      = Flatten()(block4)
    
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    output      = Activation('linear')(dense)
    
    return Model(inputs=input_main, outputs=output)


In [12]:
eegnet_model = EEGNet()

In [13]:
eegnet_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14, 641, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 14, 637, 25)       150       
                                                                 
 batch_normalization (BatchN  (None, 14, 637, 25)      100       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 14, 637, 25)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 318, 25)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 318, 25)       0     

In [14]:
eegnet_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [15]:
eegnet_model.fit(X_train, y_train, epochs=400, batch_size=16, validation_data=(X_val, y_val))

Epoch 1/400
248/248 [==============================] - 20s 23ms/step - loss: 8.9757 - accuracy: 0.2858 - val_loss: 8.1596 - val_accuracy: 0.3011
Epoch 2/400
248/248 [==============================] - 4s 17ms/step - loss: 8.2166 - accuracy: 0.3216 - val_loss: 7.9838 - val_accuracy: 0.3540
Epoch 3/400
248/248 [==============================] - 4s 17ms/step - loss: 7.6392 - accuracy: 0.3696 - val_loss: 7.5956 - val_accuracy: 0.4153
Epoch 4/400
248/248 [==============================] - 4s 17ms/step - loss: 7.3276 - accuracy: 0.4077 - val_loss: 7.0477 - val_accuracy: 0.4183
Epoch 5/400
248/248 [==============================] - 4s 17ms/step - loss: 7.0795 - accuracy: 0.4215 - val_loss: 7.1375 - val_accuracy: 0.4319
Epoch 6/400
248/248 [==============================] - 4s 17ms/step - loss: 6.8800 - accuracy: 0.4344 - val_loss: 6.8701 - val_accuracy: 0.4766
Epoch 7/400
248/248 [==============================] - 4s 17ms/step - loss: 6.7345 - accuracy: 0.4493 - val_loss: 6.9800 - val_accuracy

In [16]:
y_pred = eegnet_model.predict(X_test)

42/42 [==============================] - 1s 7ms/step


In [17]:
y_pred

array([[2.513453 , 2.1303315, 3.3699455, 6.6792765],
       [1.4406395, 1.3675815, 3.612165 , 8.771011 ],
       [2.8439906, 0.7648113, 4.7138047, 6.329714 ],
       ...,
       [1.0801206, 7.467737 , 4.70571  , 2.194997 ],
       [0.6254885, 4.672606 , 4.7438397, 4.7774506],
       [1.7059238, 4.2302547, 4.8992476, 4.628483 ]], dtype=float32)

In [18]:
y_test

array([[ 1,  1,  3,  9],
       [ 1,  1,  2,  8],
       [ 1,  1,  8,  8],
       ...,
       [ 1,  9,  4,  1],
       [ 1, 10,  1,  1],
       [ 1,  5,  7,  1]], dtype=int64)